## Bi-gram finder

Hier zoeken we naar de meest informatieve woord combinaties in de verschillende documenten. Van de top k bi-grams maken we wederom een gexf file die we nader kunnen onderzoeken met gephi.org

In [ ]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# Ngrams with a'creature' as a member
creature_filter = lambda *w: 'creature' not in w

## Bigrams
finder = BigramCollocationFinder.from_words(
   nltk.corpus.genesis.words('english-web.txt'))
# only bigrams that appear 3+ times
finder.apply_freq_filter(2)
# only bigrams that contain 'creature'
finder.apply_ngram_filter(creature_filter)
# return the 10 n-grams with the highest PMI
print(finder.nbest(bigram_measures.likelihood_ratio, 10))

In [ ]:
# RESTART here
import pandas as pd
df = pd.read_pickle("wob_data.pkl")
corpus_clean = df[['file_ids','clean_text']]


In [ ]:
corpus_clean["file_ids"]

In [ ]:
## Bigrams

bigram_corpus = []

for i in range(len(corpus_clean)):
    finder = BigramCollocationFinder.from_words(corpus_clean["clean_text"][i].split())
    # only bigrams that appear 3+ times
    finder.apply_freq_filter(1)

    bigrams_in_file = {"file_name":[],"bigrams":[]}

    # return the n-grams with the highest PMI
    bigrams_in_file["file_name"].append(corpus_clean["file_ids"][i])
    bigrams_in_file["bigrams"].append(finder.nbest(bigram_measures.likelihood_ratio, 30))
    
    bigram_corpus.append(bigrams_in_file)
    

In [ ]:
# add filename types to the graph
f=re.split('_|-',bigram_corpus[0]['file_name'][0])[0]
if len(f)<4:
    f=''
f

In [ ]:
# load networkX
import networkx as nx
import datetime
import re

# create an empty graph
Gnew=nx.Graph()

# Loop over all tweets 
for f in bigram_corpus:
        # file type
        f1=''
        #re.split(', |_|-|!', data)
        f2 = re.split('_|-|\.',f['file_name'][0])[0]
        if len(f2) > 4:
            f1 = f2
            print(f2)
    
        # Add node
        Gnew.add_node(str(f['file_name'][0]),label=f['file_name'][0], type='file'+'-'+f1)
        
        # Loop over all bigrams
        try:
            bigrams = f['bigrams'][0]
            for bg in bigrams:
                Gnew.add_node(bg[0], label=bg[0],  type="woord")
                Gnew.add_node(bg[1], label=bg[1],  type="woord")
                Gnew.add_edge(bg[0],bg[1],label="bigram")
                Gnew.add_edge(bg[0],f['file_name'][0],label="found-in")
                Gnew.add_edge(bg[1],f['file_name'][0],label="found-in")
        except:
            pass
        

In [ ]:
# save graph
nx.write_gexf(Gnew,'woberdepop.gexf')